In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Tokenizer Application").master("local[2]").getOrCreate()
sc = spark.sparkContext

25/02/05 11:32:05 WARN Utils: Your hostname, Globals-MacBook-Pro.local resolves to a loopback address: 127.94.0.1; using 172.20.10.2 instead (on interface en7)
25/02/05 11:32:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/05 11:32:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [14]:
files = sc.textFile("aclImdb/test/neg/*.txt")

In [16]:
files.count()

82

In [22]:
#files.map(lambda x: x.split(' ')).map(lambda bow: [t.lower() for t in bow]).take(2)

## Hashing Trick for Term Matrix

In [23]:
from pyspark.mllib.feature import HashingTF

In [24]:
htf = HashingTF()

In [25]:
doc = "a a b b c d".split(" ")

In [26]:
htf.transform(doc)

SparseVector(1048576, {73435: 2.0, 126124: 1.0, 465329: 1.0, 684277: 2.0})

In [27]:
bow = files.map(lambda x: x.split(' ')).map(lambda bow: [t.lower() for t in bow])

In [28]:
htf = HashingTF()

In [32]:
X = bow.map(lambda doc: htf.transform(doc))
Xb = bow.map(lambda doc: htf.setBinary(True).transform(doc))

In [33]:
X.take(1)

[SparseVector(1048576, {28095: 1.0, 34218: 1.0, 39468: 1.0, 51283: 1.0, 52764: 1.0, 68587: 1.0, 77312: 1.0, 95471: 3.0, 107871: 1.0, 122106: 1.0, 164189: 1.0, 166274: 1.0, 197668: 4.0, 209586: 1.0, 210337: 1.0, 225917: 4.0, 226524: 1.0, 249736: 1.0, 253322: 2.0, 254439: 1.0, 265377: 2.0, 278679: 1.0, 284123: 2.0, 287413: 1.0, 320232: 1.0, 320734: 1.0, 329981: 1.0, 343346: 1.0, 356129: 1.0, 377531: 3.0, 388652: 1.0, 390641: 1.0, 392803: 1.0, 413123: 1.0, 415824: 1.0, 438289: 1.0, 442177: 1.0, 445493: 1.0, 473463: 2.0, 503235: 2.0, 517606: 3.0, 528622: 1.0, 531103: 2.0, 570812: 1.0, 582719: 1.0, 597288: 1.0, 623715: 1.0, 641650: 1.0, 643579: 1.0, 676975: 1.0, 704755: 1.0, 708864: 1.0, 720433: 3.0, 727059: 3.0, 727535: 1.0, 745657: 1.0, 746242: 1.0, 747160: 2.0, 747175: 2.0, 747335: 2.0, 751977: 1.0, 754703: 1.0, 757909: 1.0, 778041: 1.0, 803534: 1.0, 817398: 1.0, 817636: 1.0, 820305: 1.0, 825338: 1.0, 856868: 1.0, 858751: 1.0, 874107: 1.0, 884498: 1.0, 892205: 1.0, 927915: 1.0, 940456: 1

In [34]:
Xb.take(1)

[SparseVector(1048576, {28095: 1.0, 34218: 1.0, 39468: 1.0, 51283: 1.0, 52764: 1.0, 68587: 1.0, 77312: 1.0, 95471: 1.0, 107871: 1.0, 122106: 1.0, 164189: 1.0, 166274: 1.0, 197668: 1.0, 209586: 1.0, 210337: 1.0, 225917: 1.0, 226524: 1.0, 249736: 1.0, 253322: 1.0, 254439: 1.0, 265377: 1.0, 278679: 1.0, 284123: 1.0, 287413: 1.0, 320232: 1.0, 320734: 1.0, 329981: 1.0, 343346: 1.0, 356129: 1.0, 377531: 1.0, 388652: 1.0, 390641: 1.0, 392803: 1.0, 413123: 1.0, 415824: 1.0, 438289: 1.0, 442177: 1.0, 445493: 1.0, 473463: 1.0, 503235: 1.0, 517606: 1.0, 528622: 1.0, 531103: 1.0, 570812: 1.0, 582719: 1.0, 597288: 1.0, 623715: 1.0, 641650: 1.0, 643579: 1.0, 676975: 1.0, 704755: 1.0, 708864: 1.0, 720433: 1.0, 727059: 1.0, 727535: 1.0, 745657: 1.0, 746242: 1.0, 747160: 1.0, 747175: 1.0, 747335: 1.0, 751977: 1.0, 754703: 1.0, 757909: 1.0, 778041: 1.0, 803534: 1.0, 817398: 1.0, 817636: 1.0, 820305: 1.0, 825338: 1.0, 856868: 1.0, 858751: 1.0, 874107: 1.0, 884498: 1.0, 892205: 1.0, 927915: 1.0, 940456: 1

## IDF for Reducing Term Importance

In [35]:
from pyspark.mllib.feature import IDF

In [40]:
idf = IDF()

In [42]:
idf.fit(Xb)

In [47]:
Xidf = idf.fit(Xb).transform(Xb)

In [49]:
Xidf.take(1)

[SparseVector(1048576, {28095: 2.8094, 34218: 3.7257, 39468: 2.0209, 51283: 3.7257, 52764: 3.7257, 68587: 2.4729, 77312: 2.8094, 95471: 0.2757, 107871: 3.7257, 122106: 1.6463, 164189: 3.0325, 166274: 3.7257, 197668: 0.0244, 209586: 3.7257, 210337: 0.0881, 225917: 0.4115, 226524: 1.123, 249736: 3.3202, 253322: 1.3278, 254439: 0.7813, 265377: 3.0325, 278679: 0.5068, 284123: 2.1163, 287413: 2.8094, 320232: 3.7257, 320734: 1.3278, 329981: 3.7257, 343346: 3.7257, 356129: 3.3202, 377531: 0.3935, 388652: 2.6271, 390641: 3.7257, 392803: 1.2833, 413123: 1.9339, 415824: 1.4231, 438289: 1.7798, 442177: 0.3584, 445493: 3.7257, 473463: 0.527, 503235: 0.4676, 517606: 0.3413, 528622: 1.5856, 531103: 2.8094, 570812: 3.7257, 582719: 2.6271, 597288: 2.6271, 623715: 3.7257, 641650: 0.8079, 643579: 3.7257, 676975: 3.7257, 704755: 2.6271, 708864: 1.4231, 720433: 0.0, 727059: 0.0244, 727535: 3.0325, 745657: 3.7257, 746242: 1.8539, 747160: 0.8925, 747175: 1.6463, 747335: 1.7108, 751977: 0.8079, 754703: 3.725